In [3]:
import time
import json
import yaml
import boto3
import base64
import logging
import asyncio
import paramiko
from utils import *
from constants import *
from scp import SCPClient
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
from botocore.exceptions import NoCredentialsError, ClientError

executor = ThreadPoolExecutor()




In [4]:
instance_id_list = []
fmbench_config_map = []


In [5]:
def load_yaml_file(file_path):
    """
    Load and parse a YAML file.

    Args:
        file_path (str): The path to the YAML file to be read.

    Returns:
        dict: Parsed content of the YAML file as a dictionary.
    """
    with open(file_path, "r") as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as error:
            print(f"Error reading the YAML file: {error}")
            return None


In [6]:
config_data = load_yaml_file(yaml_file_path)


In [7]:
print(config_data)


{'aws': {'region': 'us-east-1', 'iam_instance_profile_arn': 'arn:aws:iam::471112568442:instance-profile/EC2'}, 'run_steps': {'security_group_creation': True, 'key_pair_generation': False, 'deploy_ec2_instance': True, 'run_bash_script': True, 'delete_ec2_instance': True}, 'security_group': {'group_name': 'ec2_multi_deploy', 'description': 'MultiDeploy EC2 Security Group', 'vpc_id': None}, 'key_pair_gen': {'key_pair_name': 'ec2_multi_deploy_kp', 'key_pair_fpath': 'ec2_multi_deploy_kp.pem'}, 'instances': [{'instance_type': 'g5.2xlarge', 'ami_id': 'ami-05c3e698bd0cffe7e', 'startup_script': 'startup_scripts/gpu_ubuntu_startup.txt', 'fmbench_config': 'src/fmbench/configs/llama3/8b/config-ec2-llama3-8b.yml'}]}


In [8]:
if config_data["run_steps"]["security_group_creation"]:
    GROUP_NAME = config_data["security_group"].get("group_name")
    DESCRIPTION = config_data["security_group"].get("description", " ")
    VPC_ID = config_data["security_group"].get("vpc_id", "")
    try:
        sg_id = create_security_group(GROUP_NAME, DESCRIPTION, VPC_ID)

        if sg_id:
            # Add inbound rules if security group was created successfully
            authorize_inbound_rules(sg_id)
    except ClientError as e:
        print(f"An error occurred while creating or getting the security group: {e}")


Security Group 'ec2_multi_deploy' already exists. Fetching existing security group ID.
Inbound rule already exists for Security Group sg-013c79f5ffe82fb52. Skipping...


In [9]:
if config_data["run_steps"]["key_pair_generation"]:
    KEY_PAIR_NAME = config_data["key_pair_gen"]["key_pair_name"]
    private_key = create_key_pair(KEY_PAIR_NAME)
elif config_data["run_steps"]["key_pair_generation"] == False:
    KEY_PAIR_NAME = config_data["key_pair_gen"]["key_pair_name"]
    PRIVATE_KEY_FNAME = config_data["key_pair_gen"]["key_pair_fpath"]
    try:
        with open(f"{PRIVATE_KEY_FNAME}", "r") as file:
            private_key = file.read()
    except FileNotFoundError:
        print(f"File not found: {PRIVATE_KEY_FNAME}")
    except IOError as e:
        print(f"Error reading file {PRIVATE_KEY_FNAME}: {e}")


In [10]:
for i in config_data["instances"]:
    print(i)


{'instance_type': 'g5.2xlarge', 'ami_id': 'ami-05c3e698bd0cffe7e', 'startup_script': 'startup_scripts/gpu_ubuntu_startup.txt', 'fmbench_config': 'src/fmbench/configs/llama3/8b/config-ec2-llama3-8b.yml'}


In [11]:
# if config_data["run_steps"]["deploy_ec2_instance"]:
#     iam_arn = config_data["aws"]["iam_instance_profile_arn"]
#     print(iam_arn)
#     # WIP Parallelize This.
#     for instance in config_data["instances"]:
#         instance_type = instance["instance_type"]
#         ami_id = instance["ami_id"]
#         startup_script = instance["startup_script"]
#         # command_to_run = instance["command_to_run"]
#         with open(f"{startup_script}", "r") as file:
#             user_data_script = file.read()
#         # user_data_script += command_to_run
#         # Create an EC2 instance with the user data script
#         instance_id = create_ec2_instance(
#             KEY_PAIR_NAME,
#             sg_id,
#             user_data_script,
#             ami_id,
#             instance_type,
#             iam_arn,
#         )
#         instance_id_list.append(instance_id)
#         fmbench_config_map.append({instance_id: instance["fmbench_config"]})


In [12]:
instance_id_list.append('i-03bda4e866885b13e')
fmbench_config_map.append({'i-03bda4e866885b13e':'src/fmbench/configs/llama3/8b/config-ec2-llama3-8b.yml'})


In [13]:
def check_completion_flag(
    hostname, username, key_file_path, flag_file_path="/tmp/startup_complete.flag"
):
    """
    Checks if the startup flag file exists on the EC2 instance.

    Args:
        hostname (str): The public IP or DNS of the EC2 instance.
        username (str): The SSH username (e.g., 'ubuntu').
        key_file_path (str): The path to the PEM key file.
        flag_file_path (str): The path to the startup flag file on the instance. Default is '/tmp/startup_complete.flag'.

    Returns:
        bool: True if the flag file exists, False otherwise.
    """
    try:
        # Initialize the SSH client
        ssh_client = paramiko.SSHClient()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        # Load the private key
        private_key = paramiko.RSAKey.from_private_key_file(key_file_path)

        # Connect to the instance
        ssh_client.connect(hostname, username=username, pkey=private_key)
        print(f"Connected to {hostname} as {username}")

        # Check if the flag file exists
        stdin, stdout, stderr = ssh_client.exec_command(
            f"test -f {flag_file_path} && echo 'File exists'"
        )
        output = stdout.read().decode().strip()
        error = stderr.read().decode().strip()

        # Close the connection
        ssh_client.close()

        # Return True if the file exists, otherwise False
        return output == "File exists"

    except Exception as e:
        print(f"Error connecting via SSH to {hostname}: {e}")
        return False


In [14]:
def wait_for_flag(
    instance,
    max_wait_time=600,
    check_interval=30,
    flag_file_path="/tmp/startup_complete.flag",
):
    """
    Waits for the startup flag file on the EC2 instance, and returns the script if the flag file is found.

    Args:
        instance (dict): The dictionary containing instance details (hostname, username, key_file_path).
        formatted_script (str): The bash script content to be executed.
        remote_script_path (str): The remote path where the script should be saved on the instance.
        max_wait_time (int): Maximum wait time in seconds (default: 600 seconds or 10 minutes).
        check_interval (int): Interval time in seconds between checks (default: 30 seconds).
    """
    elapsed_time = 0
    while elapsed_time < max_wait_time:
        # Check if the startup flag exists on the instance
        startup_complete = check_completion_flag(
            hostname=instance["hostname"],
            username=instance["username"],
            key_file_path=instance["key_file_path"],
            flag_file_path=flag_file_path,
        )

        if startup_complete:
            return True

        # Wait for the specified check interval before trying again
        print(
            f"{flag_file_path} flag file not found. Checking again in {check_interval} seconds..."
        )
        time.sleep(check_interval)
        elapsed_time += check_interval

    return False


In [15]:
# if config_data["run_steps"]["deploy_ec2_instance"]:
#     instance_details = generate_instance_details(
#         instance_id_list, PRIVATE_KEY_FNAME, fmbench_config_map, region="us-east-1"
#     )
#     for instance in instance_details:
#         # Format the script with the specific config file
#         formatted_script = bash_script.format(config_file=instance["config_file"])

#         if wait_for_flag(instance):
#             print("Startup Script complete, executing fmbench now")
#             # Upload and execute the script on the instance
#             script_output = upload_and_execute_script_invoke_shell(
#                 hostname=instance["hostname"],
#                 username=instance["username"],
#                 key_file_path=instance["key_file_path"],
#                 script_content=formatted_script,
#                 remote_script_path=remote_script_path,
#             )
#     for instance in instance_details:
#         if wait_for_flag(instance, flag_file_path="/tmp/fmbench_completed.flag"):
#             print("Fmbench Run successful, Getting the folders now")
#             check_and_retrieve_results_folder(instance)


In [16]:
async def execute_fmbench(instance, formatted_script, remote_script_path):
    """
    Asynchronous wrapper for deploying an instance using synchronous functions.
    """
    # Check for the startup completion flag
    startup_complete = await asyncio.get_event_loop().run_in_executor(
        executor, wait_for_flag, instance, 600, 30, "/tmp/startup_complete.flag"
    )

    if startup_complete:
        print("Startup Script complete, executing fmbench now")

        # # Upload and execute the script on the instance
        # script_output = await asyncio.get_event_loop().run_in_executor(
        #     executor,
        #     upload_and_execute_script_invoke_shell,
        #     instance["hostname"],
        #     instance["username"],
        #     instance["key_file_path"],
        #     formatted_script,
        #     remote_script_path,
        # )
        # print(f"Script Output from {instance['hostname']}:\n{script_output}")

        # Check for the fmbench completion flag
        fmbench_complete = await asyncio.get_event_loop().run_in_executor(
            executor, wait_for_flag, instance, 1200, 30, "/tmp/fmbench_completed.flag"
        )

        if fmbench_complete:
            print("Fmbench Run successful, Getting the folders now")
            await asyncio.get_event_loop().run_in_executor(
                executor, check_and_retrieve_results_folder, instance, 'output'
            )


# Function to execute the deployment tasks in parallel
async def multi_deploy_fmbench(instance_details, bash_script, remote_script_path):
    tasks = []

    # Create a task for each instance
    for instance in instance_details:
        # Format the script with the specific config file
        formatted_script = bash_script.format(config_file=instance["config_file"])

        # Create an async task for this instance
        tasks.append(execute_fmbench(instance, formatted_script, remote_script_path))

    # Run all tasks concurrently
    await asyncio.gather(*tasks)


In [17]:
async def main():
    await multi_deploy_fmbench(instance_details, bash_script, remote_script_path)


In [18]:
if config_data["run_steps"]["deploy_ec2_instance"]:
    instance_details = generate_instance_details(
        instance_id_list, PRIVATE_KEY_FNAME, fmbench_config_map, region="us-east-1"
    )# Call the async function
    await main()


Connected to ec2-3-236-82-193.compute-1.amazonaws.com as ubuntu
Startup Script complete, executing fmbench now
Connected to ec2-3-236-82-193.compute-1.amazonaws.com as ubuntu
Fmbench Run successful, Getting the folders now
Connected to ec2-3-236-82-193.compute-1.amazonaws.com as ubuntu
Retrieving folder '/home/ubuntu/foundation-model-benchmarking-tool/results-llama3-8b-g5.2xl-ec2' from ec2-3-236-82-193.compute-1.amazonaws.com to 'output/results-llama3-8b-g5.2xl-ec2'...
Folder '/home/ubuntu/foundation-model-benchmarking-tool/results-llama3-8b-g5.2xl-ec2' retrieved successfully to 'output/results-llama3-8b-g5.2xl-ec2'.


In [19]:
instance_details


[{'hostname': 'ec2-3-236-82-193.compute-1.amazonaws.com',
  'username': 'ubuntu',
  'key_file_path': 'ec2_multi_deploy_kp.pem',
  'config_file': 'src/fmbench/configs/llama3/8b/config-ec2-llama3-8b.yml'}]

In [20]:
# if config_data["run_steps"]["delete_ec2_instance"]:
#     for instance_id in instance_id_list:
#         delete_ec2_instance(instance_id)
#     instance_id_list = []
